In [1]:
# Scinet modules
import scinet.ed_copernicus_elliptic as edc
from scinet import *

# My custom modules
import scinet.ed_stokes as sto

#Other modules
import datetime
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np

%load_ext tensorboard
import tensorflow as tf
import datetime, os

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

### Helper Functions

In [18]:
def gen_input_stokes(Fr, St, t_observ, tt_predicted):
    in1 = np.array([sto.stokes_eqn(Fr, St, t_observ) for _ in tt_predicted])
    in2 = np.reshape(tt_predicted, (-1, 1))
    out = in2 #dummy filler
    return [in1, in2, out]

def stokes_representation_plot(net, Fr_range, St_range, t_observ, step_num=100, eval_time=1.5):
    Fr_vec = np.linspace(*Fr_range, num=step_num)
    St_vec = np.linspace(*St_range, num=step_num)
    
    FR, ST = np.meshgrid(Fr_vec, St_vec)
    
    eval_time = np.array([eval_time])
    out = np.array([net.run(gen_input_stokes(Fr, St, t_observ, eval_time), net.mu)[0] for Fr, St in zip(np.ravel(FR), np.ravel(ST))])
    
    fig = plt.figure(figsize=(net.latent_size*3.9, 2.1))
    for i in range(net.latent_size):
        zs = out[:, i]
        ax = fig.add_subplot('1{}{}'.format(net.latent_size, i + 1), projection='3d')
        Z = np.reshape(zs, FR.shape)
        surf = ax.plot_surface(FR, ST, Z, rstride=1, cstride=1, cmap=cm.inferno, linewidth=0)
        ax.set_xlabel(r'$Fr$')
        ax.set_ylabel(r'$St$')
        ax.set_zlabel('Latent activation {}'.format(i + 1))
        if (i==2):
            ax.set_zlim(-1,1) #Fix the scale for the third plot, where the activation is close to zero
        ax.set_zticks([-1,-0.5,0,0.5,1])
    fig.tight_layout()
    return fig

def plot_stokes_prediction(net_, Fr_, St_, t_observ, t_predict):    
    x_correct = sto.stokes_eqn(Fr, St, t_predict)
    x_predict = net.run(gen_input_stokes(Fr_, St_, t_observ, t_predict), net.output).ravel()
    fig = plt.figure(figsize=(7, 4))
    ax = fig.add_subplot(111)
    ax.plot(t_predict, x_correct, color=orange_color, label='True time evolution')
    ax.plot(t_predict, x_predict, '--', color=blue_color, label='Predicted time evolution')
    ax.set_xlabel(r'$t$ [$s$]')
    ax.set_ylabel(r'$x$ [$m$]')
    handles, labels = ax.get_legend_handles_labels()
    lgd=ax.legend(handles, labels,loc='upper left', bbox_to_anchor=(0.6, 1.3), shadow=True, ncol=1)
    fig.tight_layout()
    return fig

## New Network

In [2]:
netName = 'stokesRNNet'
observation_size = 1
latent_size = 3
question_size = 0
answer_size = 1

dev_percent = 5
num_examples = 10000
time_series_length = 10

encoder_layout  = [500,100]
decoder_layout  = [100,100]
timestep_layout = []
myBeta     = 1e-3
myL2_coeff = 1e-3

t_sim = np.linspace(0,1,time_series_length)
print(t_sim)

[0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]


### Data creation and loading

In [8]:
temp = sto.stokes_data_recurrent(num_examples, t_sample=t_sim, fileName='stokes_example');

In [9]:
print(type(temp))
for i in temp:
    print(type(i))

<type 'tuple'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>


In [10]:
td, vd, ts, vs = dl.load(dev_percent, 'stokes_example')

### Create and train neural network

In [11]:
# Create network object
net = nn.Network(latent_size, observation_size, time_series_length, answer_size,
                 encoder_num_units=encoder_layout, decoder_num_units=decoder_layout, euler_num_units=timestep_layout,
                 name=netName) 








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where









In [14]:
# Print initial reconstruction loss (depends on initialization)
print(len(vd))

print(np.shape(vd))
print(np.shape(td))
print(net.run(vd, net.recon_loss)) #default
print(net.run(vd, net.kl_loss))
print(net.rnn_depth)

#print(net_pretrained.run(vd, net_pretrained.recon_loss)) #default
#print(net_pretrained.run(vd, net_pretrained.kl_loss))
#print(net_pretrained.rnn_depth)

500
(500, 10)
(9500, 10)
0.6604775
61689.695
9


In [15]:
log_path = os.path.join(io.tf_log_path, netName)
%load_ext tensorboard
%tensorboard --logdir /home/jrwest/Courses/Winter_2021/CS230/project/nn_physical_concepts_cs230/tf_log/StokesNet

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [16]:
# Train
num_epochs = 400
print_frequency = 0.2

check_epochs = int(print_frequency * num_epochs)
num_loops    = int(num_epochs/check_epochs)

for i in range(num_loops):
    net.train(check_epochs, 256, 0.001, td, vd, beta_fun=(lambda x: myBeta), euler_l2_coeff=myL2_coeff, test_step=10 )
    # Check progress. It is recommended to use Tensorboard instead for this.
    print(net.run(vd, net.recon_loss)) #default
    print(net.run(vd, net.kl_loss))

  0%|          | 0/80 [00:00<?, ?it/s]

0.3916747
0.010833976


  0%|          | 0/80 [00:00<?, ?it/s]

0.38839775
0.677269


  0%|          | 0/80 [00:00<?, ?it/s]

0.38833818
0.06912709


  0%|          | 0/80 [00:00<?, ?it/s]

0.38618246
0.8692244


  0%|          | 0/80 [00:00<?, ?it/s]

0.38777265
0.3669553


## Plot sample trajectory

In [22]:
# Plot prediction
%matplotlib tk
Fr = 1.0
St = 3.0
t_predict = np.linspace(0, 1, 250)
net.run()
plot_stokes_prediction(net, Fr, St, t_sim, t_predict);

TypeError: run() takes at least 3 arguments (1 given)

## Plot sample trajectory

In [ ]:
%matplotlib tk
plot_velocity_mars(series_length=1000, delta_t=.7)

## Plot of latent activations

In [ ]:
# The mean anomalies M_E and M_M of Earth and Mars, respectively.
%matplotlib tk
fig = copernicus_latent_neurons(net,series_length = 20)

### Calculate L2-norm of error

In [ ]:
data, states = edc.copernicus_data(20, 5000)

In [ ]:
np.sqrt(net.run(data, net.recon_loss))

In [ ]:
example, example_states = edc.copernicus_data(50, 1)
out = net.run(example, net.decoded_list)
out = np.reshape(out, (len(out),2))
plot_orbits(series_length=100, delta_t = 7)

In [ ]:
#Choose Network Name for saving (if desired)
date_str  = str(datetime.datetime.now().date())
name_str  = 'CoperNetcus_'
filename  = name_str + date_str
full_path = io.tf_save_path + filename

In [ ]:
#Save Network
if os.path.isfile(full_path+'.pkl'):
    print("Filename already exists. Please choose another name.")
else:    
    net.save(filename)